## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

df = spark.read.csv(file_location, header=True, inferSchema=True)

In [0]:
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
df.columns

Out[5]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [0]:
## handle categorical features using pyspark

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="sex", outputCol="sex_indexed")
df_r = indexer.fit(df).transform(df)
df_r.show(5)


+----------+----+------+------+---+------+----+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|
+----------+----+------+------+---+------+----+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|
+----------+----+------+------+---+------+----+-----------+
only showing top 5 rows



In [0]:
indexer = StringIndexer(inputCols=["smoker", "day","time"], outputCols=["smoker_indexed","day_indexed", "time_indexed"])
df_r = indexer.fit(df_r).transform(df_r)
df_r.show(5)

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
only showing top 5 rows



In [0]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["tip","sex_indexed", "smoker_indexed", "day_indexed","time_indexed"], outputCol="Independent")
output=assembler.transform(df_r)

In [0]:
output.select("Independent").show(5)

+--------------------+
|         Independent|
+--------------------+
|[1.01,1.0,0.0,1.0...|
|(5,[0,3],[1.66,1.0])|
| (5,[0,3],[3.5,1.0])|
|(5,[0,3],[3.31,1.0])|
|[3.61,1.0,0.0,1.0...|
+--------------------+
only showing top 5 rows



In [0]:
final_data = output.select("Independent", "total_bill")
final_data.show(5)

+--------------------+----------+
|         Independent|total_bill|
+--------------------+----------+
|[1.01,1.0,0.0,1.0...|     16.99|
|(5,[0,3],[1.66,1.0])|     10.34|
| (5,[0,3],[3.5,1.0])|     21.01|
|(5,[0,3],[3.31,1.0])|     23.68|
|[3.61,1.0,0.0,1.0...|     24.59|
+--------------------+----------+
only showing top 5 rows



In [0]:
from pyspark.ml.regression import LinearRegression

train_data,test_data = final_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol="Independent", labelCol="total_bill")
regressor = regressor.fit(train_data)

In [0]:
regressor.coefficients

Out[33]: DenseVector([4.1706, -0.5374, 1.9367, -0.7214, -0.4466])

In [0]:
regressor.intercept

Out[34]: 7.6865464154444725

In [0]:
pred_results = regressor.evaluate(test_data)
pred_results.predictions.show(5)

+--------------+----------+------------------+
|   Independent|total_bill|        prediction|
+--------------+----------+------------------+
|(5,[0],[1.45])|      9.55|13.733861697208692|
|(5,[0],[1.97])|     12.02|15.902554074117237|
| (5,[0],[2.0])|     12.69| 16.02767094201581|
|(5,[0],[2.31])|     18.69|17.320545243634367|
|(5,[0],[3.27])|     17.78|21.324285016388608|
+--------------+----------+------------------+
only showing top 5 rows



In [0]:
pred_results.meanAbsoluteError , pred_results.meanSquaredError

Out[37]: (4.907570763088478, 41.61206764034156)

In [0]:
# Create a view or table

temp_table_name = "tips_csv"

df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `tips_csv`

In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "tips_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)